# Video Codec Unit (VCU) Demo Example: TPG->HDR ENCODE->FILE

# Introduction

Video Codec Unit (VCU) in ZynqMP SOC is capable of encoding and decoding AVC/HEVC compressed video streams in real time.

This notebook example shows High Dynamic Range (HDR) video recording usecase – the process of capturing raw video from a software test pattern generator, encoding with HDR10 metadata using VCU and then storing its content in file. The file stored is a recorded compressed file. 

__Important Note:__
As of VLC Vetinari (Version 3.0.11), there is no AVC HDR support in VLC. Thus, if you try to play/stream out an AVC HDR file to VLC, you will not see proper HDR metadata.


# Implementation Details

<img src="pictures/block-diagram-videotestsrc-hdr-to-file.png" align="center" alt="Drawing" style="width: 400px; height: 200px"/>

### Board Setup 
1. Connect Ethernet cable.
2. Connect serial cable to monitor logs on serial console.
3. If Board is connected to private network, then export proxy settings in /home/root/.bashrc file as below,      
    - create/open a bashrc file using "vi ~/.bashrc" 
        - Insert below line to bashrc file
            - export http_proxy="< private network proxy address >"
            - export https_proxy="< private network proxy address >"
        - Save and close bashrc file.

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Run the Demo

In [2]:
from ipywidgets import interact
import ipywidgets as widgets
from common import common_vcu_demo_videotestsrc_hdr_to_file
import os
from ipywidgets import HBox, VBox, Text, Layout

### Video

In [3]:
codec_type=widgets.RadioButtons(
    options=['avc', 'hevc'],
    description='Codec Type:',
    disabled=False)
video_size=widgets.RadioButtons(
    options=['640x480', '1280x720', '1920x1080', '3840x2160'],
    description='Resolution:',
    disabled=False)
color_format=widgets.RadioButtons(
    options=['NV12_10LE32', 'NV16_10LE32'],
    description='Color Format:',
    disabled=False)
HBox([codec_type, video_size, color_format])

### HDR Metadata:

##### Display Primaries:

In [4]:
red_x=widgets.Text(value='0',
    placeholder='(optional) 5, 37000',
    description='Red X:',
    disabled=False)
red_y=widgets.Text(value='0',
    placeholder='(optional) 5, 42000',
    description='Red Y:',
    disabled=False)
green_x=widgets.Text(value='0',
    placeholder='(optional) 5, 37000',
    description='Green X:',
    disabled=False)

display(HBox([red_x, red_y, green_x]))

In [5]:
green_y=widgets.Text(value='0',
    placeholder='(optional) 5, 42000',
    description='Green Y:',
    disabled=False)
blue_x=widgets.Text(value='0',
    placeholder='(optional) 5, 37000',
    description='Blue X:',
    disabled=False)
blue_y=widgets.Text(value='0',
    placeholder='(optional) 5, 42000',
    description='Blue Y:',
    disabled=False)

display(HBox([green_y, blue_x, blue_y]))

##### White Point:

In [6]:
white_x=widgets.Text(value='0',
    placeholder='(optional) 5, 37000',
    description='White X:',
    disabled=False)
white_y=widgets.Text(value='0',
    placeholder='(optional) 5, 42000',
    description='White Y:',
    disabled=False)

display(HBox([white_x, white_y]))

##### Display Luminance:

In [7]:
max_display_luminance=widgets.Text(value='0',
    placeholder='(optional) 50000, 10000000',
    description='Max:',
    disabled=False)
min_display_luminance=widgets.Text(value='0',
    placeholder='(optional) 1, 50000',
    description='Min:',
    disabled=False)

display(HBox([max_display_luminance, min_display_luminance]))

##### Content Light Level:

In [8]:
max_cll=widgets.Text(value='0',
    placeholder='(optional) 0, 65535',
    description='Max CLL:',
    disabled=False)
max_fall=widgets.Text(value='0',
    placeholder='(optional) 0, 65535',
    description='Max FALL:',
    disabled=False)

display(HBox([max_cll, max_fall]))

### Advanced Encoder Options:

In [9]:
frame_rate=widgets.Text(value='',
    placeholder='(optional) 15, 30, 60',
    description='Frame Rate:',
    disabled=False)
bit_rate=widgets.Text(value='',
    placeholder='(optional) 1000, 20000',
    description='Bit Rate(Kbps):',
    style={'description_width': 'initial'},
    disabled=False)
gop_length=widgets.Text(value='',
    placeholder='(optional) 30, 60',
    description='Gop Length',
    disabled=False)

display(HBox([bit_rate, frame_rate, gop_length]))

In [10]:
periodicity_idr=widgets.Text(value='',
    placeholder='(optional) 30, 60',
    description=r'<p>IDR Period:</p>',
    disabled=False)
no_of_frames=widgets.Text(value='',
    placeholder='(optional) 1000, 2000',
    description=r'<p>Frame Nos:</p>',
    disabled=False)
output_path=widgets.Text(value='',
    placeholder='(optional) output.avc',
    description='Output Path:',
    disabled=False)
HBox([periodicity_idr, no_of_frames, output_path])

In [11]:
show_fps=widgets.Checkbox(
    value=False,
    description='show-fps',
    disabled=False)
HBox([show_fps])

In [12]:
from IPython.display import clear_output
from IPython.display import Javascript

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

def clear_op(event):
    clear_output(wait=True)
    return

button1 = widgets.Button(
    description='Clear Output',
    style= {'button_color':'lightgreen'},
    layout={'width': '300px'}
)
button2 = widgets.Button(
    description='',
    style= {'button_color':'white'},
    layout={'width': '83px'}
)
button1.on_click(run_all)
button1.on_click(clear_op)

In [13]:
def start_demo(event):
    #clear_output(wait=True)
    arg = [];
    arg = common_vcu_demo_videotestsrc_hdr_to_file.cmd_line_args_generator(video_size.value, codec_type.value, output_path.value, no_of_frames.value, bit_rate.value, frame_rate.value, gop_length.value, periodicity_idr.value, color_format.value, red_x.value, red_y.value, green_x.value, green_y.value, blue_x.value, blue_y.value, white_x.value, white_y.value, max_display_luminance.value, min_display_luminance.value, max_cll.value, max_fall.value, show_fps.value);
    !sh vcu-demo-videotestsrc-hdr-to-file.sh $arg
    return

button = widgets.Button(
    description='click to start videotestsrc-hdr-to-file demo',
    style= {'button_color':'lightgreen'},
    layout={'width': '300px'}
)
button.on_click(start_demo)
HBox([button, button2, button1])

# References
[1] https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18842546/Xilinx+Video+Codec+Unit

[2] https://www.xilinx.com/support.html#documentation (Refer to PG252)